## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

## Import Packages

In [2]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

## Camera calibration using chessboard images

In [2]:
# Step 1: extract object points and image points
# prepare object points, like (0,0,0),(1,0,0),(2,0,0),...,(nx,ny,0)
nx = 9
ny = 6
objp = np.zeros((nx*ny,3), np.float32)
objp[:,:2] = np.mgrid[0:nx,0:ny].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (nx,ny), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (nx,ny), corners, ret)
        #cv2.imshow('img',img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

# do camera calibration using object points and image points
# test undistortion on first calibration image and save undistorted image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = img.shape[1::-1]


ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

undst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('output_images/test_undist.jpg', undst)

True

## Pipeline for combined color and gradient thresholds

### Gradient Thresholds using sobel

In [3]:
# Define a function that takes an image, gradient orientation,
# and threshold min / max values.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0,255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # use if image is read in via mpimg
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # use if image is read in via cv2
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value
    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel))
    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel))
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # using exclusive (>, <) thresholds
    binary_output[(scaled_sobel > thresh[0]) & (scaled_sobel < thresh[1])] = 1

    # Return the result
    return binary_output

# Define a function to return the magnitude of the gradient
# for a given sobel kernel size and threshold values
def mag_thresh(img, sobel_kernel=3, thresh=(0, 255)):
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # use if image is read in via mpimg
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # use if image is read in via cv2
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255
    gradmag = (gradmag/scale_factor).astype(np.uint8)
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag > thresh[0]) & (gradmag < thresh[1])] = 1

    # Return the binary image
    return binary_output

# Define a function to threshold an image for a given range and Sobel kernel
def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)  # use if image is read in via mpimg
    #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # use if image is read in via cv2
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction,
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1

    # Return the binary image
    return binary_output

### Combined Sobel thresholds

In [6]:
# read image
image = mpimg.imread('test_images/test5.jpg')

# Choose a Sobel kernel size
ksize_abs = 3 # Choose a larger odd number to smooth gradient measurements
ksize_mag = 9 # Choose a larger odd number to smooth gradient measurements
ksize_dir = 15 # Choose a larger odd number to smooth gradient measurements

# defines thresholds
thresh_abs_x = (30, 180)
thresh_abs_y = (30, 180)
thresh_mag = (70, 120)
thresh_dir = (np.pi/3, np.pi/2)

# Apply each of the thresholding functions
gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize_abs, thresh=thresh_abs_x)
grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize_abs, thresh=thresh_abs_y)
mag_binary = mag_thresh(image, sobel_kernel=ksize_mag, thresh=thresh_mag)
dir_binary = dir_threshold(image, sobel_kernel=ksize_dir, thresh=thresh_dir)

combined = np.zeros_like(dir_binary)
a = True
#combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
#combined[((gradx == 1) & (grady == 1))] = 1
#combined[((gradx == 1))] = 1
#combined[((gradx == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
#combined[((grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
combined[(((gradx == 1) & a) & (grady == 1)) | ((mag_binary == 1))] = 1
#combined[((gradx == 1) & (grady == 1)) | ((dir_binary == 1))] = 1                          #5

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined, cmap='gray')
ax2.set_title('Thresholded Magnitude', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

### Color thresholds

In [11]:
# Define a function that thresholds the H-, L- and S-channel of HLS
def hls_select(img, thresh=(0, 255), chan = 's'):
    # convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    # extract channels
    h_channel = hls[:,:,0]
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # channel selection
    if chan=='h':
        channel = h_channel
    if chan=='l':
        channel = l_channel
    if chan=='s':
        channel = s_channel
    # apply threshold
    binary_output = np.zeros_like(channel)
    binary_output[(channel > thresh[0]) & (channel <= thresh[1])] = 1
    return binary_output

image = mpimg.imread('test_images/test5.jpg')

thresh_hls = (90,140)

hls_binary = hls_select(image, thresh_hls, chan='s')

combined_grad_hls = np.zeros_like(dir_binary)
combined_grad_hls[(combined == 1) | (hls_binary == 1)] = 1

# Plot the result
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(combined_grad_hls, cmap='gray')
ax2.set_title('Thresholded Magnitude', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

In [12]:
# Parameters
thresh_dict = {
    "thresh_abs_x"  : (30, 180),            # threshold for sobel operator in x direction
    "thresh_abs_y"  : (30, 180),            # threshold for sobel operator in y direction
    "thresh_dir"    : (np.pi/3, np.pi/2),   # threshold for sobel operator considering gradient direction
    "thresh_mag"    : (70, 120),            # threshold for sobel operator considering gradient magnitude
    "thresh_h"      : (1,255),              # threshold for h channel
    "thresh_l"      : (1,255),              # threshold for l channel
    "thresh_s"      : (90, 140)             # threshold for s channel
}

operatoracv_dict = {
    "abs_x_acv" : True,     # activate thresholding for sobel operator in x direction
    "abs_y_acv" : True,     # activate thresholding for sobel operator in y direction
    "dir_acv"   : False,    # activate thresholding for sobel operator considering gradient direction
    "mag_acv"   : True,     # activate thresholding for sobel operator considering gradient magnitude
    "hls_acv"   : True      # activate thresholding for hls channels
}

kernel_dict = {
    "kernel_abs"    : 3,    # kernel size for sobel operator gradient absolute value
    "kernel_dir"    : 15,   # kernel size for sobel operator gradient direction
    "kernel_mag"    : 9     # kernel size for sobel operator gradient magnitude
}

In [6]:

# Pipeline Thresholding
def pipeline_thresholds(img, thresh_dict, operatoracv_dict, kernel_dict, hls_channel="s"):
    img = np.copy(img)
    # Convert to HLS color space and separate the
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel
    if orient == 'x':
        sobel_raw = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    if orient == 'y':
        sobel_raw = cv2.Sobel(l_channel, cv2.CV_64F, 0, 1) # Take the derivative in y
    abs_sobel = np.absolute(sobel_raw) # Absolute x derivative to accentuate lines from horizontal / vertical
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel)) # rescale to 8 bit int

    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1

    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    color_binary = np.dstack(( np.zeros_like(sxbinary), sxbinary, s_binary)) * 255
    return color_binary